In [4]:
%load_ext autoreload
%autoreload 2

import wandb
import text3d2video.wandb_util as wbu
from omegaconf import OmegaConf
import torch

torch.cuda.is_available()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


False

In [ ]:
from text3d2video.evaluation.video_comparison import VideoLabel, make_comparison_vid, scene_key_fun
from text3d2video.util import group_list_by

api = wandb.Api()

filters = {"tags": "generative_rendering_ablation"}
runs = list(api.runs("romeu/diffusion-3D-features", filters=filters))

def sort_fun(run):
    cfg = OmegaConf.create(run.config)
    no_uv = not cfg.generative_rendering.do_uv_noise_init
    no_pre_attn = not cfg.generative_rendering.do_pre_attn_injection
    no_post_attn = not cfg.generative_rendering.do_post_attn_injection 

    if no_uv:
        return 3
    elif no_pre_attn:
        return 2
    elif no_post_attn:
        return 1
    else:
        return 0

# group runs by scene
runs_grouped = group_list_by(runs, scene_key_fun)

# sort runs in each scene consistently
for runs in runs_grouped:
    runs.sort(key=sort_fun)

def info_fun(run):

    label = VideoLabel() 
    cfg = OmegaConf.create(run.config)

    no_uv = not cfg.generative_rendering.do_uv_noise_init
    no_pre_attn = not cfg.generative_rendering.do_pre_attn_injection
    no_post_attn = not cfg.generative_rendering.do_post_attn_injection 

    if no_uv:
        label.content = "No UV initialization"
    elif no_pre_attn:
        label.content = "No pre attn injection"
    elif no_post_attn:
        label.content = "No post attn injection"
    else:
        label.content = "Full"

    return label

vid = make_comparison_vid(
    runs_grouped,
    info_fun,
    download=False,
    title="Generative Rendering Ablation",
    show_guidance_video=False,
)

vid.ipython_display()

wandb: Downloading large artifact rumba:v3, 144.70MB. 73 files... 
wandb:   73 of 73 files downloaded.  
Done. 0:0:0.1
